レース環境情報をスクレイピングで取得

In [44]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import datetime

In [2]:
race_id = "202104020101"
url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
html = requests.get(url)
html.encoding= "EUC_JP"
soup = BeautifulSoup(html.text, "html.parser")

In [3]:
日付 = soup.find("dd",class_='Active')
日付.find("a").get_text()

'5月8日(土)'

In [4]:
レース情報 = soup.find("div", class_="RaceList_Item02") 

In [5]:
レース名 = レース情報.find("div",class_="RaceName").get_text()
re.findall(r"\w+",レース名)

['4歳以上障害OP']

In [6]:
レース環境 = レース情報.find("div",class_="RaceData01").get_text()
re.findall(r"\w+",レース環境)

['09', '50発走', '障2890m', '芝', '外', '内', '天候', '晴', '馬場', '良']

In [7]:
df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
race_id_list

Index(['202101010101', '202101010102', '202102010101', '202102010102',
       '202103010101', '202103010102', '202104010101', '202104010102',
       '202105010101', '202105010102',
       ...
       '202109040102', '202109040201', '202109040202', '202109040301',
       '202110040101', '202110040102', '202110040201', '202110040202',
       '202110040301', '202110040302'],
      dtype='object', length=173)

df.pickleに保存したdataframeのindexを参照してして全てのレース環境情報をスクレイピング

In [8]:
race_infos = {}
for race_id in tqdm(race_id_list):
  url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
  html = requests.get(url)
  html.encoding= "EUC_JP"
  soup = BeautifulSoup(html.text, "html.parser")

  日付 = soup.find("dd",class_='Active')
  日付 = 日付.find("a").get_text()
  レース情報 = soup.find("div", class_="RaceList_Item02") 
  # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
  # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
  レース環境 = レース情報.find("div",class_="RaceData01").get_text()
  info = re.findall(r"\w+",レース環境)
  info[0] = info[2][:1]
  info[1] = info[2][1:]
  info.pop(2)
  info_dict = {}
  # print(info)
  info_dict["date"] = f"{race_id[:4]}年{日付[:-3]}"

  for text in info:
    # print(text)
    if text in ["芝","ダ"]:
      # print("芝付き")
      info_dict["race_type"] = text
    if "障" in text:
      info_dict["race_type"] = "障害"
    if "m" in text:
      # print("mつき")
      info_dict["course_ren"] = re.findall(r"\d+",text)[0]
    if text in ["雨","小雨","小雪","雪","晴","曇"]:
      # print("天気")
      info_dict["weather"] = text
    # if "年" in text:
    #   info_dict["data"] = text
    if text in ["良","不良","重","稍","不"]:
      # print("馬場")
      info_dict["ground_state"] = text


  race_infos[race_id] = info_dict
  time.sleep(.5)

100%|██████████| 173/173 [02:35<00:00,  1.11it/s]


In [19]:
race_infos_df = pd.DataFrame(race_infos).T
race_infos_df

,date,race_type,course_ren,weather,ground_state
202101010101,2021年6月12日,芝,1200,小雨,良
202101010102,2021年6月12日,ダ,1700,曇,良
202102010101,2021年7月3日,芝,1200,晴,良
202102010102,2021年7月3日,ダ,2400,晴,良
202103010101,2021年7月3日,芝,2750,曇,良
...,...,...,...,...,...
202110040102,2021年8月14日,芝,1200,雨,不
202110040201,2021年8月15日,芝,1200,晴,重
202110040202,2021年8月15日,ダ,1700,晴,不
202110040301,2021年8月21日,芝,1200,曇,良


In [20]:
results = pd.read_pickle("df.pickle")
results

,着順,枠,馬番,馬名,斤量,騎手,人気,性,齢,体重,体重変化
202101010101,1,5.0,9,ディーバサンライズ,52.0,△秋山稔,7.0,牝,3,474,-22
202101010101,2,3.0,6,バンベルク,56.0,横山和,2.0,牡,3,470,-2
202101010101,3,7.0,14,リツィタル,54.0,大野,14.0,牝,3,394,2
202101010101,4,5.0,10,クレマチステソーロ,54.0,鮫島駿,5.0,牝,3,468,2
202101010101,5,3.0,5,ペイシャケイティー,54.0,古川吉,4.0,牝,3,456,8
...,...,...,...,...,...,...,...,...,...,...,...
202110040302,12,3.0,5,メイショウヤマツミ,56.0,小牧,6.0,牡,3,500,-18
202110040302,13,3.0,6,サンデーインアスク,54.0,川田,4.0,牝,3,414,0
202110040302,14,5.0,9,ベントラ,56.0,酒井,12.0,牡,3,500,6
202110040302,15,7.0,13,ギレルモ,56.0,岩田望,10.0,牡,3,490,0


In [21]:
results_addinfo = results.merge(race_infos_df,left_index=True,right_index=True,how="inner")
results_addinfo

,着順,枠,馬番,馬名,斤量,騎手,人気,性,齢,体重,体重変化,date,race_type,course_ren,weather,ground_state
202101010101,1,5.0,9,ディーバサンライズ,52.0,△秋山稔,7.0,牝,3,474,-22,2021年6月12日,芝,1200,小雨,良
202101010101,2,3.0,6,バンベルク,56.0,横山和,2.0,牡,3,470,-2,2021年6月12日,芝,1200,小雨,良
202101010101,3,7.0,14,リツィタル,54.0,大野,14.0,牝,3,394,2,2021年6月12日,芝,1200,小雨,良
202101010101,4,5.0,10,クレマチステソーロ,54.0,鮫島駿,5.0,牝,3,468,2,2021年6月12日,芝,1200,小雨,良
202101010101,5,3.0,5,ペイシャケイティー,54.0,古川吉,4.0,牝,3,456,8,2021年6月12日,芝,1200,小雨,良
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202110040302,12,3.0,5,メイショウヤマツミ,56.0,小牧,6.0,牡,3,500,-18,2021年8月21日,ダ,1700,曇,良
202110040302,13,3.0,6,サンデーインアスク,54.0,川田,4.0,牝,3,414,0,2021年8月21日,ダ,1700,曇,良
202110040302,14,5.0,9,ベントラ,56.0,酒井,12.0,牡,3,500,6,2021年8月21日,ダ,1700,曇,良
202110040302,15,7.0,13,ギレルモ,56.0,岩田望,10.0,牡,3,490,0,2021年8月21日,ダ,1700,曇,良


In [22]:
results_addinfo.columns

Index(['着順', '枠', '馬番', '馬名', '斤量', '騎手', '人気', '性', '齢', '体重', '体重変化', 'date',
       'race_type', 'course_ren', 'weather', 'ground_state'],
      dtype='object')

In [43]:
pd.set_option('display.max_rows',None)
# results_addinfo["ground_state"].value_counts()
results_addinfo.isnull().sum()


着順              0
枠               0
馬番              0
馬名              0
斤量              0
騎手              0
人気              0
性               0
齢               0
体重              0
体重変化            0
date            0
race_type       0
course_ren      0
weather         0
ground_state    0
dtype: int64

馬番の型

日付データの成形

In [48]:
results_addinfo["date"] = pd.to_datetime(results_addinfo["date"],format="%Y年%m月%d日")
results_addinfo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2416 entries, 202101010101 to 202110040302
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            2416 non-null   int64         
 1   枠             2416 non-null   float64       
 2   馬番            2416 non-null   object        
 3   馬名            2416 non-null   object        
 4   斤量            2416 non-null   float64       
 5   騎手            2416 non-null   object        
 6   人気            2416 non-null   float64       
 7   性             2416 non-null   object        
 8   齢             2416 non-null   int64         
 9   体重            2416 non-null   int64         
 10  体重変化          2416 non-null   int64         
 11  date          2416 non-null   datetime64[ns]
 12  race_type     2416 non-null   object        
 13  course_ren    2416 non-null   object        
 14  weather       2416 non-null   object        
 15  ground_state  2416 non-n